# Justin Butler
## AAE 537: HW 4

This assignment was created in Jupyter Notebook and formatted via $\LaTeX$

In [1]:
%%javascript
// Making sure the outputs display correctly
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Importing required packages for this homework
import numpy as np
import matplotlib.pyplot as plt
#from IPython.display import HTML, display
#import tabulate
%matplotlib inline

def rounds(item):
    ret = float('%.3f'% (item * 1000/1000))
    return(ret)

### i)
For the first part of this assignment, I will be labeling out how to find the stagnation temperature at each station in the engine as well as the fuel/air ratios in the burner and afterburner. We will assume that q, $\beta$, CPR, $C_p$, $\Delta H_B$ (of the fuel), $\dot{m}_0$, T$_{t4}$, and T$_{t7}$ are all known terms. We will also assume we know the efficiencies of the engine, that is, we know $\eta_c, \eta_b, \eta_t, \eta_{ab},$ and $\eta_n$.

#### Inlet
For the inlet, we will use the use the atmospheric conditions at a given altitude as given by the atmospheric model NASA provides at https://www.grc.nasa.gov/WWW/k-12/airplane/atmosmet.html. 

From static pressure we can also find the mach. Assuming $\gamma=1.33$ (or that $\gamma$ is at least a known value) and assuming the ambient air acts like an ideal gas we know:
$$M_0 = \sqrt{\frac{2\cdot q}{\gamma\cdot P_{s0}}}$$

Now with static temperature and mach, we find total temperature at the inlet as:
$$T_{t0} = T_{s0} \left(1 + \frac{\gamma-1}{2}M_0^2\right)$$
From this, we will assume an adiabatic intake - thus $T_{t0} = T_{t2}$. Next, by using the inlet recovery per Mil Std 5008B we can define the piecewise function for pressure ratio as:
$\frac{P_{t2}}{P_{t0}} = \begin{cases}
  1 & \text{for }M_0 <  1\\
  1 - 0.075(M_0-1)^{1.35} & \text{for } 1< M_0 < 5\\
  \frac{800}{M_0^4+935} & \text{for } M_0 > 5 
\end{cases}$

Since stagnation pressure can be calculated as the sum of static and dynamic pressures, we find $P_{t0} = P_{s0} + q$ where $P_{s0}$ is the pressure found from the atmospheric model. From this we can find the total pressure at station 2 from $P_{t2} = P_{t0} \cdot \left(P_{t2}/P_{t0}\right)$.
#### Compressor

For the compressor we can find the stagnation temperature and stagnation pressure for the air by:
$$T_{t3} = T_{t2}\left[1+\frac{1}{\eta_c}\cdot\left(CPR^{(\gamma-1)/\gamma}-1\right)\right]$$
$$P_{t3} = P_{t2}\cdot CPR$$

We will also want to find the massflow running through the core. Since we know $\beta$ and the inlet massflow $\dot{m}$, the massflow going through the core would be:
$$ \dot{m}_c = \frac{\dot{m}}{\beta+1}$$
#### Burner

Moving onto the burner, $T_{t4}$ is a given value and we will assume that $P_{t4} = P_{t3}$. We can find the fuel to air ratio by:
$$f = \frac{T_{t4} - T_{t3}}{\Delta H_B}\cdot C_p$$
#### Turbine

Station 5, within the figure, looks like it occurs before the re-mixing of the core and bypass streams. Thus, we will assume that the massflow out of the core at stage 5 is $\dot{m}_c + \dot{m}_f$. With this value, we will use powerbalance between the compressor and the turbine to determine the total temperature at the end of the turbine. Indeed:
$$T_{t5} = T_{t4} - \frac{\dot{m}_c\cdot C_p\cdot(T_{t3}-T_{t2})}{(\dot{m}_c+\dot{m}_f)\cdot C_p}$$
And from this, we can use isentropic relations to find:
$$P_{t5} = P_{t4}\left(\frac{T_{t5}}{T_{t4}}\right)^\frac{\gamma}{(\gamma-1)}$$
#### Mixer

Looking at the figure, by the time we reach station 6 the bypass flow has rejoined. Thus, the massflow now includes the bypass flow which we will need to account for. For this, we will perform weighted averages (thus assuming a constant area mixer).
$$T_{t6} = \frac{\dot{m}_b}{\dot{m}_c+\dot{m}_f+\dot{m}_b}\cdot T_{t2} + \frac{\dot{m}_c + \dot{m}_f} {\dot{m}_c+\dot{m}_f+\dot{m}_b} \cdot T_{t5}$$
$$P_{t6} = \frac{\dot{m}_b}{\dot{m}_c+\dot{m}_f+\dot{m}_b}\cdot P_{t2} + \frac{\dot{m}_c + \dot{m}_f} {\dot{m}_c+\dot{m}_f+\dot{m}_b} \cdot P_{t5}$$
#### Afterburner

At the afterburner, $T_{t7}$ is a given value and we will again assume that $P_{t7}=P_{t6}$. We will now perform an energy balance. Assuming a constant $C_p$ we can derive the following:
\begin{align*}
(\dot{m}_b+\dot{m}_c+\dot{m}_f+\dot{m}_{AB})C_pT_{t7} &= (\dot{m}_b+\dot{m}_c+\dot{m}_f)C_pT_{t6} + \dot{m}_{AB}\Delta H_b\eta_{ab}\\
\left(\beta + 1 + f + \frac{\dot{m}_{AB}}{\dot{m}_c}\right)C_pT_{t7} &= (\beta + 1 + f)C_pT_{t6} + \frac{\dot{m}_{AB}}{\dot{m}_c}\Delta H_b\eta_{AB}\\
\frac{\dot{m}_{AB}}{\dot{m}_c} &= \frac{(\beta + 1 + f)C_pT_{t6} - (\beta + 1 + f)C_pT_{t7}}{C_pT_{t7}-\Delta H_b\eta_{AB}}
\end{align*}
At this point, we will assume that the combuster is performing at stoichiometric conditions and thus the only air coming into the afterburner was air that went through the bypass. Thus:
$$f_{ab} = \frac{\dot{m}_{AB}}{\dot{m}_b}$$
Thus, we take the derived relation and multiply each side by $\beta^{-1}$ to obtain:
$$f_{ab} = \frac{(\beta + 1 + f)C_pT_{t6} - (\beta + 1 + f)C_pT_{t7}}{C_pT_{t7}-\Delta H_b\eta_{AB}} \cdot \beta^{-1}$$

#### Nozzle

A perfectly expanded nozzle means that $P_9 = P_0$ and that $P_{t9} = P_{t7}$. We will also assume this is an adiabatic process and thus $T_{t9} = T_{t7}$. We can then find the velocity at the exit by:
$$V_9 = \sqrt{2C_pT_{t9}\eta_n\left(1-\left(\frac{P_9}{P_{t9}}\right)^\frac{\gamma-1}{\gamma}\right)}$$


### ii)
The question here asked us to create a script for running an engine analysis. In order to do this, the first step is to create methods for the atmospheric model and the isentropic relations. These are below:

In [ ]:
# Atmospheric model created using the NASA equation found at:
# https://www.grc.nasa.gov/WWW/k-12/airplane/atmosmet.html
class atmosphere:
    def __init__(self, val, valGiven = 0,units = "SI"):
        #Convert from US to SI
        if units != "SI" and valGiven == 0:
            val = val / 3.281
        if units != "SI" and valGiven == 1:
            val = val * 0.04788
        #0 implies the given value is an altitude
        #1 implies the given value is a pressure
        if valGiven == 0:
            self.h = val
        elif valGiven == 1:
            self.P = val
        else:
            print("Not a valid 'valGiven' parameter.")
    def hCalc(self):
        if self.h < 11000:
            self.T = 15.04 - 0.00649*self.h
            self.P = 101.29 * ((self.T + 273.1)/288.08)**(5.256)
        elif self.h < 25000:
            self.T = -56.46
            self.P = 22.65 * np.exp(1.73 - 0.000157*self.h)
        elif self.h > 24999:
            self.T = -131.21 + 0.00299*self.h
            self.P = 2.488 * ((self.T + 273.1)/216.6)**(-11.388)
        self.rho = self.P / (0.2869 * (self.T + 273.1))
    def PCalc(self):
        if self.P > 22.632:
            self.T = (288.08*(self.P/101.29)**(1/5.256))-273.1
            self.h = (self.T - 15.04)/(-0.00649)
        elif self.P > 0.1113586:
            self.T = -56.46
            self.h = (1.73 - np.log((self.P/22.65)))/(0.000157)
        else:
            self.T = (216.6*(self.P/2.488)**(1/(-11.388)))-273.1
            self.h = (self.T + 131.21)/0.00299
        self.rho = self.P / (0.2869 * (self.T + 273.1))
# ISENTROPIC RELATIONS
def totalPres(Ps, M,gamma=1.4):
    total = Ps * ((1+((gamma-1)/2)*M**2)**(gamma/(gamma-1)))
    return(total)
def statPres(Pt, M, gamma=1.4):
    static = Pt / ((1+((gamma-1)/2)*M**2)**(gamma/(gamma-1)))
    return(static)
def totalTemp(Ts, M, gamma=1.4):
    total = Ts * ((1+((gamma-1)/2)*M**2))
    return(total)
def statTemp(Tt, M, gamma=1.4):
    static = Tt / ((1+((gamma-1)/2)*M**2))
    return(static)     

***MODEL***

With those methods defined, we can now construct a Python Class that runs an engine analysis. The created class includes a help file which was printed to the console to show that this script runs correctly.

In [175]:
###################################################
# Engine class constructed from the above analysis#
###################################################
class engine:
    def help():
        print("Engine Class Help File:")
        print("The engine class takes as input several parameters pertaining to a \n"
             "gas turbine engine and calculates information from those starting parameters.\n"
             "The required inputs are in the following order:\n"
              "1. Altitude\n"
              "2. dynamic pressure\n"
              "3. Flow Bypass Ratio \n"
              "4. Compressor Pressure Ratio \n"
              "5. ratio of specific heats\n"
              "6. Total temperature at end of combustor\n"
              "7. Total temperature at exit of the afterburner\n"
              "8. A 5 element list containing the efficiences (c,b,t,ab,n)\n"
              "9. Heating value of the fuel\n"
              "10. Mass flow through the inlet"
             )
        print("\nThese values are set in the class with default values for this problem.\n"
              "Alternatively, one can run with 'manualEntry = True' to be able to enter\n"
              "these values manually when the object is created.\n"
              " \n"
              "There are also several manual hard-coded entries with this method. These hard\n"
              "coded segments are used for the rest of the assignment for which this method\n"
              "was constructed. You can ignore them for general use.\n"
              "\n"
              "There is also a seperate method within this class that allows you to convert\n"
              "you're inputs to SI units if they were not originally. Using the command\n"
              "'.changeToSI()' will perform the conversions from US to SI.\n"
              "\n"
            "Running the '.runAnalysis()' command will generate all of the desired values. \n"
            "These include values of temperatures, pressures, and initial mach. There will \n"
              "also be values calculated for the specific thrust and fuel consumption.\n")
        print("\nFor errors in this code, you can contact Justin on Github as user BananaBonanza")
    def __init__(self,manualEntry = False,
                altitude = 0, q = 1500, BPR = 0,
                 CPR = 20, gamma = 1.33, Cp = 0.3,
                 Tt4 = 3000, Tt7 = 3700, nc = 0.88,
                 nb = 0.95, nt = 0.9, nab = 0.9,
                 nn = 0.9,Hb = 18500, mDot = 500
                ):
        if manualEntry:
            inputTerms = ["altitude", "dynamic pressure", "Flow Bypass Ratio", 
                      "compressor Pressure Ratio","ratio of specific heats",
                      "total temperature at end of combustor",
                      "total temperature at exit of the afterburner",
                      "the different efficiences",
                      "Heating value of the fuel", "Mass flow through inlet"]
            inputVal = []
            for item in inputTerms:
                print("Input the value of " + item)
                if item != "the different efficiences":
                    inputVal.append(float(input()))
                else:
                    effTerms = ["Compressor: ", "Burner: ",
                                "Turbine: ", "Afterburner: ",
                                "Nozzle: "]
                    effVal = []
                    for jtem in effTerms:
                        print("  " + jtem)
                        eff = input()
                        print(eff)
                        effVal.append(eff)
                    inputVal.append(effVal)
            print(inputVal)
            self.setValues(inputVal)
        self.alt = altitude
        self.q = q
        self.BPR = BPR
        self.CPR = CPR
        self.gamma = gamma
        self.Cp = Cp
        self.Tt4 = Tt4
        self.Tt7 = Tt7
        self.nc = nc
        self.nb = nb
        self.nt = nt
        self.nab = nab
        self.nn = nn
        self.Hb = Hb
        self.mDot = mDot
        self.R = 287. #[J/kg K]
        self.Rus = 1717. #[ft lbf/slug °R]
    def changeToSI(self):
        self.alt = self.alt * 0.3048
        self.q = 1500 * 47.880258888889 #Pa
        self.Cp = 0.3 * 4186.
        self.Tt4 = ((self.Tt4 - 32) * (5/9)) + 273.15
        self.Tt7 = ((self.Tt7 - 32) * (5/9)) + 273.15
        self.Hb = 18500 * 1055.06 * (1/0.453592)
        self.mDot = 500 * 0.453592 #kg/s
    def setValues(self,inputVal):
        print("This code not yet supported")
        return null
    def runAnalysis(self,hardCode=0,afterburner=1):
        if hardCode == 0:
            #Atmospheric Conditions
            atmos = atmosphere(self.alt)
            atmos.hCalc()
            self.P0 = atmos.P
            #Stage 0 and Inlet (stage 1)
            self.rho0 = atmos.rho
            self.M0 = np.sqrt((self.q * 2)/(self.gamma * self.P0))
            self.Pt0 = self.P0 + self.q
            self.T0 = atmos.T
        elif hardCode == 3:
            self.M0 = 1
            self.P0 = 101325 #Pa
            self.T0 = 288.15 #K
        elif hardCode == 4:
            self.M0 = 4
            self.P0 = (2*self.q)/(self.gamma * self.M0**2)
            atmos = atmosphere(self.P0,1)
            atmos.PCalc()
            self.T0 = atmos.T
            self.h = atmos.h
        elif hardCode == 5:
            self.M0 = 2
            self.P0 = (2*self.q)/(self.gamma * self.M0**2)
            atmos = atmosphere(self.P0,1)
            atmos.PCalc()
            self.T0 = atmos.T
            self.h = atmos.h
        if hardCode != 0:
            self.Pt0 = totalPres(self.P0,self.M0,self.gamma)
        self.Rgas = self.Cp*(1-self.gamma**(-1))
        self.Tt0 = totalTemp(self.T0,self.M0,self.gamma)
        #Stage 2
        if self.M0 < 1:
            self.Pt2 = self.Pt0
        elif self.M0 < 5:
            self.Pt2 = self.Pt0*( 1 - 0.075*(self.M0 - 1)**(1.35))
        else:
            self.Pt2 = self.Pt0 * ((800/(self.M0**4 + 935)))
        self.Tt2 = self.Tt0
        #Compressor
        self.Tt3 = self.Tt2 * (1 + 
                              (1/self.nc) *
                              (self.CPR**((self.gamma-1)/self.gamma)-1))
        self.Pt3 = self.Pt2 * self.CPR
        self.mDotc = self.mDot/(self.BPR + 1)
        self.mDotb = self.mDot - self.mDotc
        #Burner
        self.f = ((self.Tt4-self.Tt3)/self.Hb)*self.Cp
        self.mDotf = self.mDotc * self.f
        self.Pt4 = self.Pt3
        #Turbine
        #self.Tt5 = self.Tt4 - ((self.mDotc * self.Cp * (self.Tt3-self.Tt2))/
        #                       ((self.mDotc + self.mDotf) * self.Cp)
        #                      )
        self.Pt5 = (self.Pt4 * 
                    (1 -(
                        ((self.Pt3/self.Pt2)**((self.gamma-1)/self.gamma) - 1)/
                        (self.nt*self.nc *(self.Tt4/self.Tt2))
                        )
                    )**(self.gamma/(self.gamma-1))
                   )
        self.Tt5 = (self.Tt4 * 
                   (1 - self.nt*(1 - (self.Pt5/self.Pt4))
                   )**((self.gamma-1)/self.gamma)
                   )
        #Mixer
        self.mDotMix = self.mDotc + self.mDotf + self.mDotb
        self.Tt6 = ((self.mDotb/self.mDotMix)*self.Tt2
                    +((self.mDotc + self.mDotf)/self.mDotMix)*self.Tt5)
        self.Pt6 = ((self.mDotb/self.mDotMix)*self.Pt2
                   + ((self.mDotc + self.mDotf)/self.mDotMix)*self.Pt5)
        if afterburner == 1:
            #Afterburner
            self.Pt7 = self.Pt6
            self.fab = (((self.BPR+1+self.f)*self.Cp*self.Tt6 -
                        (self.BPR+1+self.f)*self.Cp*self.Tt7
                        )/
                        (self.Cp*self.Tt7 - self.Hb * self.nab)
                       ) * self.BPR**(-1)
            self.mDotfab = self.mDotMix * self.fab
            #Nozzle
            self.P9 = self.P0
            self.Pt9 = self.Pt7
            self.Tt9 = self.Tt7
        else:
            self.P9 = self.P0
            self.Pt9 = self.Pt6
            self.Tt9 = self.Tt6
            self.fab = 0
            self.mDotfab = 0
        self.V9 = np.sqrt(2 * self.Cp * self.Tt9 * self.nn *
                          (1 - 
                           (self.P9/self.Pt9)**((self.gamma-1)/self.gamma))
                         )
        #Final values
        self.fuelFlow = self.mDotf + self.mDotfab
        self.mDotEXIT = self.fuelFlow + self.mDot
        
        #self.spcThrust = ((self.mDotMix + self.mDotfab)*self.V9 -
        #                  self.mDot * (self.M0 * np.sqrt(self.gamma * self.R * self.T0))
        #                 )
        self.F = ((self.mDot)*(1+self.f+self.fab)*self.V9 -
                  self.mDot * (self.M0 * np.sqrt(self.gamma * self.R * self.T0))
                 )
        self.spcThrust = (self.F/self.mDot)
        self.F = self.spcThrust*(self.mDotMix+self.mDotfab)
        self.TSFC = self.fuelFlow / self.F
        
        
engine.help()

Engine Class Help File:
The engine class takes as input several parameters pertaining to a 
gas turbine engine and calculates information from those starting parameters.
The required inputs are in the following order:
1. Altitude
2. dynamic pressure
3. Flow Bypass Ratio 
4. Compressor Pressure Ratio 
5. ratio of specific heats
6. Total temperature at end of combustor
7. Total temperature at exit of the afterburner
8. A 5 element list containing the efficiences (c,b,t,ab,n)
9. Heating value of the fuel
10. Mass flow through the inlet

These values are set in the class with default values for this problem.
Alternatively, one can run with 'manualEntry = True' to be able to enter
these values manually when the object is created.
 
There are also several manual hard-coded entries with this method. These hard
coded segments are used for the rest of the assignment for which this method
was constructed. You can ignore them for general use.

There is also a seperate method within this class tha

### iii)
Determining thrust and TSFC for $M = 1.0$, $BPR = 0$, and $CPR = 20$. Also with $M_4 = 0.4$ we will find the flow area at the entrance of the turbine.

One part of this problem required us to find the molecular mass of air and kerosene. The molecular mass we use for air is $28.97$ kg/kmol. We then approximated kerosene as dodecane and found a molar mass of $171.48$ kg/kmol.

Also, we found that for a $M=1$ condition, the altitude given from the atmospheric model gave a negative values. Thus, we will not use the given $q$ value and instead assume $P_0 = 101.325$ kPa and that $q$ is whatever it needs to make the relation work.

In [182]:
engine3 = engine()
engine3.changeToSI()
engine3.runAnalysis(hardCode=3,afterburner=0)
print("V9: ",engine3.V9, "m/s")
print("TSFC: ", rounds(engine3.TSFC*10**5),"E-5", "Kg/Ns")
print("Thrust: ",rounds(engine3.F), "Newtons")
R = engine3.Cp*(1-engine3.gamma**(-1))
T5 = statTemp(engine3.Tt5,0.4,engine3.gamma)
P5 = statPres(engine3.Pt5,0.4,engine3.gamma)
V5 = 0.4 * np.sqrt(engine3.gamma * R * T5)
A = (((engine3.mDot+engine3.mDotf) * R * T5)/
     (P5 * V5))
print("Area: ",rounds(A),"square meters")

V9:  1259.7897916360791 m/s
TSFC:  3.388 E-5 Kg/Ns
Thrust:  227709.507 Newtons
Area:  0.315 square meters


### iv)
Setting BPR $ = 0.5$, $M_b = 0.5$. We will use methods discussed in the previous homework when we dealt with constant pressure mixers. The idea is we want to find a CPR that finds the area at the exit of the Turbine to match what we found in step 3. To find this, since we know the mach of the bypass we will do the following process:

1) We will guess a CPR.

2) Find $P_2$ from isentropic relations since we know $M_b$.

3) Since we are assuming $P_2=P_5$, we can solve the isentropic relation for the turbine exit to get:
$$M_5 = \sqrt{\left(\left(\frac{P_t}{P_s} \right)^\frac{\gamma-1}{\gamma}-1 \right)\cdot\frac{2}{\gamma-1}}$$

4) From this mach, we then use the mass flow equation to find the Area. We use the equation:

$$A = \frac{\sqrt{T_t}}{P_t}*\sqrt{\frac{R}{\gamma}}M^{-1}\left(1 + \frac{\gamma-1}{2}M^2\right)^{-\frac{\gamma+1}{2(gamma-1)}}$$

5) If this area matches our area from part (3) then we are good. Otherwise we go back to step 1 and guess a new CPR.

6) Once we find the correct CPR, we can find the static pressure, static temperature, and velocity of the bypass. Then, assuming an ideal gas, find the area of the bypass by:
$$A = \frac{\gamma\cdot R\cdot T}{P\cdot V}$$

In [180]:
i=0
cprGuess = 1
print("A",A)
while True:
    i +=1
    engine4=engine(CPR = cprGuess,BPR=0.5)
    engine4.changeToSI()
    engine4.runAnalysis(hardCode=4)
    P2 = statPres(engine4.Pt2,0.5,engine4.gamma)
    M5 = np.sqrt(((engine4.Pt5/P2)**((engine4.gamma-1)/engine4.gamma)-1)*
                (2/(engine4.gamma-1))
                )
    Aguess = ((engine4.mDotc + engine4.mDotf) * (np.sqrt(engine4.Tt2)/engine4.Pt2) *
              np.sqrt(engine4.R/engine4.gamma) * (1/M5) * 
              (1 + ((engine4.gamma-1)/2)*M5**2)**((engine4.gamma+1)/(2*(engine4.gamma - 1)))
             )
    if np.abs(Aguess - A) < 0.001:
        print("Aguess", Aguess)
        print("CPR: ", cprGuess)
        break
    if i > 100000 or cprGuess < 0:
        print("overflow")
        break
    cprGuess += 0.001
print("--- Values ---")
print("Thrust: ",rounds(engine4.F), "N")
print("TSFC: ",rounds(engine4.TSFC * 10**5), "E-5 kg/Nsf")

A 0.3148834596728165
Aguess 0.3139898060374523
CPR:  2.3849999999998475
--- Values ---
Thrust:  143404.182 N
TSFC:  14.368 E-5 kg/Nsf


In [169]:
#Finding bypass area
mDotBy = 500/3 * 0.453592 #kg/s
R = 287
Tb = statTemp(engine4.Tt2,0.5,engine4.gamma)
Pb = statPres(engine4.Pt2,0.5,engine4.gamma)
Vb = 0.5 * np.sqrt(engine4.gamma * R * Tb)
Ab = ((mDotBy * R * Tb)/
      (Pb * Vb))
print("Area of the bypass: ",rounds(Ab),"square meters")

Area of the bypass:  0.114 square meters


### v)
Now, we set $M_0 = 2$. The process we will use is as follows.

1) We will specify a BPR guess that we will work from.

2) From that BPR ratio we can obtain a mass flow for the core and the bypass

3) We next need to set up the massflow equation so it can be solved for mach. To do this, we will the equation:
$$\frac{AP_t}{\sqrt{T_t}}\sqrt{\frac{\gamma}{R}}M\left(1+\frac{\gamma-1}{2}M^2\right)^\frac{\gamma+1}{2(\gamma-1)} - \dot{m} = 0$$
to be able try to find the mach. We will guess a Mach value and see if this equation indeed equals zero. If it doesn't, we guess a new Mach until the value converges. This method will be used for both the Mach of the bypass flow and the mach of the core flow at the exit of the turbine.

4) We then generate a value for $P_2$, that is, the static pressure of the bypass. We do this using isentropic relations.

5) A CPR will be specified - iterated through the different BPR values

6) The CPR will give us $T_{t5}$ and $P_{t5}$ which will use to find $M_5$ as in step 3.

7) We use isentropic relations to find $P_5$.

8) If $P_5 = P_2$, then we are done. Otherwise, we will choose a new CPR or a new BPR until the values converge.

***NOTE***

The method as it was originally coded was not converging correctly. Thus, the code was altered to find the minimum value of $|P_5 - P_2|$ instead of where the two were equal. This gave a minimum difference of approximately $15$ Pa, which is fairly close to equality of these two value. To attempt to get fully equal pressure flow was not able to be fully constructed.

In [195]:
##               Mass flow Choking from:                  ##
## https://www.grc.nasa.gov/WWW/K-12/airplane/mflchk.html ##
def machFromMassFlow(mDot,A,Pt,Tt,gamma,R):
    MGuess = 1
    testVal = (((A * Pt)/np.sqrt(Tt))*np.sqrt(gamma/R)*MGuess*
               (1 + ((gamma-1)/2)*MGuess**2)**(-(gamma-1)/(2*(gamma-1))))-mDot
    i = 0
    while testVal > 0:
        MGuess -= 0.01
        testVal = (((A * Pt)/np.sqrt(Tt))*np.sqrt(gamma/R)*MGuess*
               (1 + ((gamma-1)/2)*MGuess**2)**(-(gamma-1)/(2*(gamma-1))))-mDot
        i += 1
        if i > 10000 or MGuess < 0:
            #print("overflow mach")
            break
    return(MGuess)
BPRguess = 0.1
CPRguess = 1.1
i = 0
j = 0
Pdifi = []
jtem = []
bprIndex= []
while BPRguess < 1:
    #print(BPRguess)
    j = 0
    CPRguess = 1
    Pdifj = []
    while CPRguess < 30:
        engine5= engine(BPR = BPRguess, CPR=CPRguess,mDot=500)
        engine5.changeToSI()
        engine5.runAnalysis(hardCode=5)
        #Assuming beta
        M2 = machFromMassFlow(engine5.mDotb,Ab,engine5.Pt2,engine5.Tt2,engine5.gamma,engine5.R)
        #print("M2: ",M2)
        P2 = statPres(engine5.Pt2,M2,engine5.gamma)
        #print("P2: ",P2)
        #Assuming CPR
        M5 = machFromMassFlow(engine5.mDotc+engine5.mDotf,A,engine5.Pt5,
                              engine5.Tt5,engine5.gamma,engine5.Rgas)
        #print("M5: ",M5)
        P5 = statPres(engine5.Pt5,M5,engine5.gamma)
        #print("P5: ",P5)
        if np.abs(P2-P5) != 0:
            Pdifj.append(np.abs(P2-P5))
        if np.abs(P2-P5)<1:
            #print("match!")
            #print("BPR: ",rounds(BPRguess))
            #print("CPR: ",rounds(CPRguess))
            #print(np.abs(P2-P5))
            #i = 100000
            break
        #EndStep
        CPRguess += 0.1
        j += 1
        if j > 500:
            #print("overflow j")
            break
    if Pdifj != []:
        if min(Pdifj) != 0:
            jtem.append(Pdifj.index(min(Pdifj)))
            Pdifi.append(min(Pdifj))
            bprIndex.append(BPRguess)
            
    BPRguess += 0.001
    i += 1
    if i > 500:
        #print("overflow i")
        break
i = Pdifi.index(min(Pdifi))
print("CPR: ",1+0.1*jtem[i])
print("BPR: ",bprIndex[i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:191: RuntimeWarning: invalid value encountered in sqrt


CPR:  6.6000000000000005
BPR:  0.23600000000000013


In [200]:
#Taking the values from above and generating the engine
engine5 = engine(BPR = 0.236, CPR = 6.6)
engine5.changeToSI()
engine5.runAnalysis(hardCode=5)
print("Thrust: ",rounds(engine5.F),"N")
print("SFC: ",rounds(engine5.TSFC*10**5),"E-5 N*s/m")


Thrust:  217280.522 N
SFC:  21.086 E-5 N*s/m
